# Dev Notebook for dataloader

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import os
import glob
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

#import ray
#ray.init()

#from result_utils.results_utils import xml2csv_path

#import matplotlib.pyplot as plt


#from result_utils.dotdict import dotdict
#from matplotlib.collections import LineCollection 

## Select experiment

In [4]:
scenario = 'motorway'
experiment_name = 'incident1'
path = f'../{scenario}/Results/{experiment_name}'

result_folders = os.listdir(path)
print(f'{len(result_folders)} simulations')

simulation_folders = [folder for folder in result_folders if folder.startswith(experiment_name)]

380 simulations


In [5]:
# Check for uncaught errors 
error_folders = []
for folder in simulation_folders:
    folder_dir = os.listdir(f'{path}/{folder}')
    if len(folder_dir) != 11:
        print(f'{folder} is missing files it has only {len(folder_dir)}')
        error_folders.append(folder)
    
    error_logs = [file for file in glob.glob(f'{path}/{folder}/*.err')]
    for log in error_logs:
        with open(log, 'r') as read_obj:
            one_char = read_obj.read(1)
            if one_char:
                print(f'{folder} has error in {log}')
                error_folders.append(folder)
                

# Load data

### Load simulation data

In [15]:
from data_utils.data_loader_utils import transform_df_to_tensor

In [16]:
incident_settings_arr = []
for simulation in tqdm(simulation_folders[:10]):
    simulation_path = f'{path}/{simulation}'
    with open(f'{simulation_path}/incident_settings.json') as f:
        incident_settings = json.load(f)
        incident_settings_arr.append(incident_settings)
    

    

100%|████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 25842.91it/s]


In [17]:
incident_tables_arr = []

In [18]:
%timeit
incident_tables_arr = []
for simulation in tqdm(simulation_folders[:10]):
    simulation_path = f'{path}/{simulation}'
    results_incident = pd.read_csv(f'{simulation_path}/detectordata.csv', sep=';')
    if len(results_incident.columns) == 10:
        incident_tables_arr.append(transform_df_to_tensor(results_incident))



100%|███████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.15s/it]


In [19]:
counterfactual_tables_arr = []
for simulation in tqdm(simulation_folders[:10]):
    simulation_path = f'{path}/{simulation}'
    counterfactual_incident = pd.read_csv(f'{simulation_path}/detectordata_counterfactual.csv', sep=';')
    if len(counterfactual_incident.columns) == 10:
        incident_tables_arr.append(transform_df_to_tensor(counterfactual_incident))
    else:
        print(f'{simulation} is missing rows')

100%|███████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.14s/it]


In [22]:
incident_settings_arr[0].keys()

dict_keys(['slow_zone', 'lc_zone', 'lc_prob_zone', 'slow_zone_speed', 'run_num', 'is_incident', 'is_random', 'random_seed', 'edge', 'lanes', 'pos', 'start_time', 'start_step', 'duration_time', 'duration_steps'])

In [26]:
for i, settings in zip(incident_tables_arr, incident_settings_arr):
    print(f'from {i[0,0,0,0]} to {i[0,0,-1,0]} w. incident at {settings["start_time"]}')
    print(f'{i[0,0,60,0]}')

from 18824.0 to 27224.0 w. incident at 20024
20024.0
from 73896.0 to 82216.0 w. incident at 75096
75096.0
from 55391.0 to 63911.0 w. incident at 56591
56591.0
from 15163.0 to 23383.0 w. incident at 16363
16363.0
from 62457.0 to 70917.0 w. incident at 63657
63657.0
from 30586.0 to 39046.0 w. incident at 31786
31786.0
from 82344.0 to 86784.0 w. incident at 83544
83544.0
from 47012.0 to 55492.0 w. incident at 48212
48212.0
from 1397.0 to 9637.0 w. incident at 2597
2597.0
from 75716.0 to 84076.0 w. incident at 76916
76916.0


Do the dataframe conversion for the counterfactual as well
Make sure the order of edges is always the same

Create mask for what data padding
Cut data at the incident time
create lane feature dataframe
Create the output
Wrape in pytorch lightning datamodule

Send git and update to Rose


../motorway/Results/incident1/incident1_380 is a good example of why the counter factual might be needed to create the labels as we have congestion coming without an incident in the end of the data.

